In [16]:
import pandas as pd
import numpy as np

In [22]:
def binomial_transition(n, prob, m):
    return np.random.binomial(n, prob, m)

def poisson_transition(n, rate, m):
    return np.random.poisson(np.nan_to_num(n * rate), m)

transition_b = lambda n, prob, m: binomial_transition(n, prob, m)
transition_p = lambda n, rate, m: poisson_transition(n, rate, m)


model_settings            = {}
model_settings["m"]       = 100
model_settings["num_pop"] = 2

N0 = np.array([[1e3], [1.5e3]]) * np.ones((model_settings["num_pop"], model_settings["m"]))
C0 = 0.05 * np.ones((model_settings["num_pop"], model_settings["m"]))


In [23]:
m       = model_settings["m"]
num_pop = model_settings["num_pop"]

N       = N0
C       = C0 * N

a2c    = np.zeros((num_pop, m))
inc    = np.zeros((num_pop, m))

x0     = np.array([C, a2c, inc]) # [num_state_vars x num_pop x m]

# the movement model

M is a matrix of [np, np] and C and N vectors of size [np, m].

Mij = individuals moving for i to j.

1. Colonized ppl moving outside unit i. The people moving outside of unit i are $\sum_j M_{ij}$.

    $C_{i}^{out}= \sum_j M_{ij} \frac{C_i}{N_i}$

    In code it will be
    ```
    Cout_i = np.sum(M[i, :]) * C[i, :]/N[i, :]
    ```
    So for putting all the people in a matix it will be
    ```
    Cout = np.sum(M, axis=1, keepdims=True) * C / N
    ```



I want a matrix operator

In [71]:
M = np.random.uniform(size=(num_pop, num_pop))


In [85]:
Cout = np.sum(M, axis=1, keepdims=True)  * np.clip(np.nan_to_num(C/N), 0, 1)


(2, 100)

In [39]:
x = x0
# process

C = x[0, :, :]           # col in all populations
S = np.clip(N - C, 0, N) # sus in all populations

In [61]:
np.clip(np.nan_to_num(C[i, :] / N[i, :]),0,1)

array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05])

In [60]:
M[i,j] * C[i, :] / N[i, :]

array([0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934204,
       0.02934204, 0.02934204, 0.02934204, 0.02934204, 0.02934